Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [94]:
WORKSPACE_PATH = 'Tensorflow\workspace'
SCRIPTS_PATH = 'Tensorflow\scripts'
APIMODEL_PATH = 'Tensorflow\models'
ANNOTATION_PATH = WORKSPACE_PATH+'\\annotations'
IMAGE_PATH = WORKSPACE_PATH+'\images'
MODEL_PATH = WORKSPACE_PATH+'\models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'\pre-trained-models'
CONFIG_PATH = MODEL_PATH+'\my_fast_rcnn\pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'\my_fast_rcnn\\'

print(CHECKPOINT_PATH)

Tensorflow\workspace\models\my_fast_rcnn\


# 1. Create Label Map

In [2]:
labels = [{'name':'kardus', 'id':1}, {'name':'kaca', 'id':2}, {'name':'kertas', 'id':3}, {'name':'plastik', 'id':4}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [113]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow\workspace\annotations/train.record
Successfully created the TFRecord file: Tensorflow\workspace\annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [6]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  86% (2615/3024)
Updating files:  87% (2631/3024)
Updating files:  88% (2662/3024)
Updating files:  89% (2692/3024)
Updating files:  90% (2722/3024)
Updating files:  91% (2752/3024)
Updating files:  92% (2783/3024)
Updating files:  93% (2813/3024)
Updating files:  94% (2843/3024)
Updating files:  95% (2873/3024)
Updating files:  96% (2904/3024)
Updating files:  97% (2934/3024)
Updating files:  98% (2964/3024)
Updating files:  99% (2994/3024)
Updating files: 100% (3024/3024)
Updating files: 100% (3024/3024), done.


In [6]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [7]:
CUSTOM_MODEL_NAME = 'my_faster_rcnn' 

In [72]:
!mkdir {'Tensorflow\workspace\models/'+CUSTOM_MODEL_NAME}
# !copy f"{PRETRAINED_MODEL_PATH+'/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}"
COPY_PATH = f"{PRETRAINED_MODEL_PATH}/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/pipeline.config' {MODEL_PATH}/{CUSTOM_MODEL_NAME}"
print(COPY_PATH) 
!copy "%s".format(COPY_PATH)
# print(f"{PRETRAINED_MODEL_PATH}/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/pipeline.config {MODEL_PATH}/{CUSTOM_MODEL_NAME}")

Tensorflow\workspace\pre-trained-models\faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8\pipeline.config' Tensorflow\workspace\models\my_faster_rcnn


A subdirectory or file Tensorflow\workspace\models\my_faster_rcnn already exists.


The system cannot find the file specified.


# 5. Update Config For Transfer Learning

In [97]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [102]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [103]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [104]:
config

{'model': faster_rcnn {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "faster_rcnn_inception_resnet_v2_keras"
     batch_norm_trainable: true
   }
   first_stage_anchor_generator {
     grid_anchor_generator {
       height_stride: 16
       width_stride: 16
       scales: 0.25
       scales: 0.5
       scales: 1.0
       scales: 2.0
       aspect_ratios: 0.5
       aspect_ratios: 1.0
       aspect_ratios: 2.0
     }
   }
   first_stage_box_predictor_conv_hyperparams {
     op: CONV
     regularizer {
       l2_regularizer {
         weight: 0.0
       }
     }
     initializer {
       truncated_normal_initializer {
         stddev: 0.009999999776482582
       }
     }
   }
   first_stage_nms_score_threshold: 0.0
   first_stage_nms_iou_threshold: 0.699999988079071
   first_stage_max_proposals: 300
   first_stage_localization_loss_weight: 2.0
   first_stage_objectness_loss_weight: 1.0


In [105]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [106]:
pipeline_config.model.faster_rcnn.num_classes = 4
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'\faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8\checkpoint\ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '\label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '\train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '\label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '\test.record']

In [107]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [82]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow\models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow\workspace\models/my_faster_rcnn --pipeline_config_path=Tensorflow\workspace\models/my_faster_rcnn/pipeline.config --num_train_steps=5000


# 7. Load Train Model From Checkpoint

In [2]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [9]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [10]:
import cv2 
import numpy as np

In [11]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [105]:
cap.release()

In [12]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [42]:
detections = detect_fn(input_tensor)

In [67]:
from matplotlib import pyplot as plt